<a href="https://colab.research.google.com/github/NikhithaU/Projects/blob/main/Text_Generation_FinetuningWikipedia_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=892b4e53aade017be55bbe6e8c650e0b380c7ecc6b5c6377466c46a139829b5b
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [ ]:
import wikipedia
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

# Load Wikipedia articles
topics = ['Artificial intelligence', 'Deep learning', 'Natural language processing']
texts = []
for topic in topics:
    article = wikipedia.page(topic)
    texts.append(article.content)

# Load GPT-2 tokenizer and model
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize texts and prepare for fine-tuning
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()
for epoch in range(3):
    for batch in loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device=device)
        attention_mask = batch[1].to(device=device)
        labels = input_ids.clone().detach()
        labels[labels == tokenizer.pad_token_id] = -100
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

# Generate some text using the fine-tuned model
prompt = "Deep Learning is"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device=device)
output = model.generate(input_ids, max_length=100, do_sample=True)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Deep Learning is a new approach to building intelligent neural networks that can understand information in ways that let the user experience more than just language learning.

One of the big ideas in Artificial intelligence is to help human scientists solve problems by translating neural systems' information into the user's own computer. That gives them the power to create novel computer programs, such as machine-readable text books as computers, machines that run on a computer's memory, which can be programmed to solve problems without knowledge of the hardware


In [ ]:
model.save_pretrained('/content/sample_data/1')

Fine tuning single sentence